<a href="https://colab.research.google.com/github/nazmulhasan77/Parallel-Processing-Using-MPI-CUDA/blob/main/Searching_using_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Searching using CUDA

## Phonebook Searching including id
Dataset :"1","FATEMA JAHAN TAMMY","015 05 040"


In [ ]:
%%writefile search_phonebook_id.cu
#include <bits/stdc++.h>
#include <cuda.h>

using namespace std;

struct Contact {
  char id[50];
  char name[50];
  char number[50];
};

__device__ bool check(char* str1, char* str2, int len) {
  for(int i = 0; str1[i] != '\0'; i++) {
    int j = 0;
    while(str1[i+j] != '\0' && str2[j] != '\0' && str1[i+j] == str2[j]) {
      j++;
    }
    if(j == len-1) {
      return true;
    }
  }
  return false;
}

__global__ void searchPhonebook(Contact* phonebook, int num_contacts, char* search_name, int name_length) {
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  if(idx < num_contacts) {
    if(check(phonebook[idx].name, search_name, name_length)) {
      printf("%s %s %s\n", phonebook[idx].id, phonebook[idx].name, phonebook[idx].number);
    }
  }
}

int main(int argc, char* argv[]) {
  if(argc != 3) {
    cerr << "Usage: " << argv[0] << " <search_name> <num_threads>" << endl;
    return 1;
  }

  string search_name = argv[1];
  int num_threads = atoi(argv[2]);
  // Mount Google Drive and copy the location
  //string file_name = "/content/sample_data/phonebook1.txt";
  string file_name = "phonebook_id.txt";

  vector<Contact> phonebook;

  ifstream file(file_name);
  if(!file.is_open()) {
    cerr << "Error opening file: " << file_name << endl;
    return 1;
  }
  else {
    Contact contact;
    string line;
    int count = 0;
    while(getline(file, line)) {

      if(count>100) break;
      count++;

      // Format: "id","name","phone_number"
      int pos = line.find(",");
      strcpy(contact.id, line.substr(1, pos-2).c_str());
      line = line.substr(pos+1);
      pos = line.find(",");
      strcpy(contact.name, line.substr(1, pos-2).c_str());
      strcpy(contact.number, line.substr(pos+2, line.size()-pos-4).c_str());
      phonebook.push_back(contact);


      /* Format: "name","phone_number"

      int pos = line.find(",");
      // Extract name (without the quotes)
      strcpy(contact.name, line.substr(1, pos - 2).c_str());

      // Extract number (also without quotes)
      strcpy(contact.number, line.substr(pos + 2, line.size() - pos - 4).c_str());

      phonebook.push_back(contact);

      */
    }
    file.close();
  }
  int num_contacts = phonebook.size();
  Contact* device_phonebook;
  cudaMalloc((void**)&device_phonebook, sizeof(Contact)*num_contacts);
  cudaMemcpy(device_phonebook, phonebook.data(), sizeof(Contact)*num_contacts, cudaMemcpyHostToDevice);

  int name_length = search_name.length() + 1;
  char* device_search_name;
  cudaMalloc((void**)&device_search_name, name_length);
  cudaMemcpy(device_search_name, search_name.c_str(), name_length, cudaMemcpyHostToDevice);

  for(int i = 0; i < num_contacts; i += num_threads) {
    int thread_count = min(num_contacts-i, num_threads);
    searchPhonebook<<<1, thread_count>>>(device_phonebook + i, thread_count, device_search_name, name_length);
    cudaDeviceSynchronize();
  }

  cudaFree(device_phonebook);
  cudaFree(device_search_name);

  return 0;
}

Overwriting search_phonebook_id.cu


In [ ]:
!nvcc -arch=sm_75 search_phonebook_id.cu -o search_phonebook

In [ ]:
!time ./search_phonebook SAHA 100 > output_id.txt


real	0m0.128s
user	0m0.013s
sys	0m0.107s


In [ ]:
!time ./search_phonebook AKTER 1000 > output_id.txt


real	0m0.201s
user	0m0.013s
sys	0m0.182s


## Phonebook Searching
Dataset: "FATEMA JAHAN TAMMY","015 05 040"

### Normal Searching

In [ ]:
%%writefile search_phonebook.cu
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <cstdlib>
#include <cstring>
#include <cuda_runtime.h>
using namespace std;

#define MAX_STR_LEN 50

// CUDA kernel to perform substring check
__device__ bool check(char* str1, char* str2, int len) {
    for(int i = 0; str1[i] != '\0'; i++) {
        int j = 0;
        while(str1[i+j] != '\0' && str2[j] != '\0' && str1[i+j] == str2[j]) {
            j++;
        }
        // If we matched the full length of the search string
        if(j == len-1) {
            return true;
        }
    }
    return false;
}

// Updated kernel using flattened arrays instead of a struct
__global__ void searchPhonebook(char* d_names, char* d_numbers, int num_contacts, char* search_name, int name_length) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx < num_contacts) {
        // Calculate the starting pointer for the current contact's name and number
        char* current_name = d_names + (idx * MAX_STR_LEN);
        char* current_number = d_numbers + (idx * MAX_STR_LEN);

        if(check(current_name, search_name, name_length)) {
            printf("%s %s\n", current_name, current_number);
        }
    }
}

int main(int argc, char* argv[]) {
    if(argc != 3) {
        cerr << "Usage: " << argv[0] << " <search_name> <num_threads>" << endl;
        return 1;
    }

    string search_string = argv[1];
    int num_threads = atoi(argv[2]);
    string file_name = "phonebook.txt";

    // 1. Read all lines into a vector
    vector<string> raw_lines;
    ifstream file(file_name);
    if(!file.is_open()) {
        cerr << "Error opening file: " << file_name << endl;
        return 1;
    }

    string line;
    while(getline(file, line)) {
        if(!line.empty()) raw_lines.push_back(line);
    }
    file.close();

    int num_contacts = raw_lines.size();

    // 2. Prepare flattened host memory (Structure of Arrays approach)
    // We use a flat array where every 50 bytes represents one name/number
    char* h_names = (char*)malloc(num_contacts * MAX_STR_LEN * sizeof(char));
    char* h_numbers = (char*)malloc(num_contacts * MAX_STR_LEN * sizeof(char));

    for(int i = 0; i < num_contacts; i++) {
        string current_line = raw_lines[i];
        int pos = current_line.find(",");

        // Extract Name and Number from CSV format: "Name","Number"
        string name = current_line.substr(1, pos - 2);
        string number = current_line.substr(pos + 2, current_line.size() - pos - 3);

        // Copy strings into the flat buffer at specific offsets
        strncpy(h_names + (i * MAX_STR_LEN), name.c_str(), MAX_STR_LEN - 1);
        strncpy(h_numbers + (i * MAX_STR_LEN), number.c_str(), MAX_STR_LEN - 1);

        // Ensure null termination
        h_names[(i * MAX_STR_LEN) + (MAX_STR_LEN - 1)] = '\0';
        h_numbers[(i * MAX_STR_LEN) + (MAX_STR_LEN - 1)] = '\0';
    }

    // 3. Allocate and Copy to Device
    char *d_names, *d_numbers, *d_search_name;
    int name_len = search_string.length() + 1;

    cudaMalloc((void**)&d_names, num_contacts * MAX_STR_LEN);
    cudaMalloc((void**)&d_numbers, num_contacts * MAX_STR_LEN);
    cudaMalloc((void**)&d_search_name, name_len);

    cudaMemcpy(d_names, h_names, num_contacts * MAX_STR_LEN, cudaMemcpyHostToDevice);
    cudaMemcpy(d_numbers, h_numbers, num_contacts * MAX_STR_LEN, cudaMemcpyHostToDevice);
    cudaMemcpy(d_search_name, search_string.c_str(), name_len, cudaMemcpyHostToDevice);

    // 4. Launch Kernel
    // We handle the data in chunks based on num_threads as per your original logic
    for(int i = 0; i < num_contacts; i += num_threads) {
        int thread_count = min(num_contacts - i, num_threads);

        // Pass the pointers shifted by the current offset 'i'
        searchPhonebook<<<1, thread_count>>>(
            d_names + (i * MAX_STR_LEN),
            d_numbers + (i * MAX_STR_LEN),
            thread_count,
            d_search_name,
            name_len
        );
        cudaDeviceSynchronize();
    }

    // 5. Cleanup
    free(h_names);
    free(h_numbers);
    cudaFree(d_names);
    cudaFree(d_numbers);
    cudaFree(d_search_name);

    return 0;
}


Writing search_phonebook.cu


In [ ]:
!nvcc -arch=sm_75 search_phonebook.cu -o search_phonebook

In [ ]:
!time ./search_phonebook AKTER 100 > output1.txt


real	0m0.224s
user	0m0.014s
sys	0m0.206s


In [ ]:
!time ./search_phonebook AKTER 1000 > output1.txt


real	0m0.218s
user	0m0.011s
sys	0m0.197s


### Searching & **Sorting Ascensing** orders

In [ ]:
%%writefile search_phonebook_sort.cu
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <cstdlib>
#include <cstring>
#include <algorithm>

#include <cuda_runtime.h>
using namespace std;

#define MAX_STR_LEN 50

// Helper struct for sorting on the CPU
struct ResultContact {
    string name;
    string number;

    // Logic for ascending sort by name
    bool operator<(const ResultContact& other) const {
        return name < other.name;
    }
};

// Substring check logic
__device__ bool check(char* str1, char* str2, int len) {
    for(int i = 0; str1[i] != '\0'; i++) {
        int j = 0;
        while(str1[i+j] != '\0' && str2[j] != '\0' && str1[i+j] == str2[j]) {
            j++;
        }
        if(j == len-1) {
            return true;
        }
    }
    return false;
}

// Kernel writes a '1' to the results array if a match is found
__global__ void searchPhonebook(char* d_names, int num_contacts, char* search_name, int name_length, int* d_results) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx < num_contacts) {
        char* current_name = d_names + (idx * MAX_STR_LEN);
        if(check(current_name, search_name, name_length)) {
            d_results[idx] = 1; // Mark as found
        } else {
            d_results[idx] = 0; // Not found
        }
    }
}

int main(int argc, char* argv[]) {
    if(argc != 3) {
        cerr << "Usage: " << argv[0] << " <search_name> <num_threads>" << endl;
        return 1;
    }

    string search_string = argv[1];
    int num_threads = atoi(argv[2]);
    string file_name = "phonebook.txt";

    // Vectors to hold raw data for processing
    vector<string> host_names_vec;
    vector<string> host_numbers_vec;

    ifstream file(file_name);
    if(!file.is_open()) {
        cerr << "Error opening file: " << file_name << endl;
        return 1;
    }

    string line;
    while(getline(file, line)) {
        if(line.empty()) continue;
        int pos = line.find(",");
        string name = line.substr(1, pos - 2);
        string number = line.substr(pos + 2, line.size() - pos - 3);
        host_names_vec.push_back(name);
        host_numbers_vec.push_back(number);
    }
    file.close();

    int num_contacts = host_names_vec.size();

    // Flatten data for CUDA
    char* h_names = (char*)malloc(num_contacts * MAX_STR_LEN);
    int* h_results = (int*)malloc(num_contacts * sizeof(int));

    for(int i = 0; i < num_contacts; i++) {
        strncpy(h_names + (i * MAX_STR_LEN), host_names_vec[i].c_str(), MAX_STR_LEN - 1);
        h_names[(i * MAX_STR_LEN) + (MAX_STR_LEN - 1)] = '\0';
    }

    // Allocate Device Memory
    char *d_names, *d_search_name;
    int *d_results;
    int search_len = search_string.length() + 1;

    cudaMalloc((void**)&d_names, num_contacts * MAX_STR_LEN);
    cudaMalloc((void**)&d_results, num_contacts * sizeof(int));
    cudaMalloc((void**)&d_search_name, search_len);

    cudaMemcpy(d_names, h_names, num_contacts * MAX_STR_LEN, cudaMemcpyHostToDevice);
    cudaMemcpy(d_search_name, search_string.c_str(), search_len, cudaMemcpyHostToDevice);

    // Launch Kernel (Using grid/block logic based on num_threads)
    int blocks = (num_contacts + num_threads - 1) / num_threads;
    searchPhonebook<<<blocks, num_threads>>>(d_names, num_contacts, d_search_name, search_len, d_results);
    cudaDeviceSynchronize();

    // Copy results back to host
    cudaMemcpy(h_results, d_results, num_contacts * sizeof(int), cudaMemcpyDeviceToHost);

    // Filter matched contacts into a vector for sorting
    vector<ResultContact> matched_contacts;
    for(int i = 0; i < num_contacts; i++) {
        if(h_results[i] == 1) {
            matched_contacts.push_back({host_names_vec[i], host_numbers_vec[i]});
        }
    }

    // Sort the results in ascending order
    sort(matched_contacts.begin(), matched_contacts.end());

    // Print final sorted output
    cout << "Search Results (Ascending Order):" << endl;
    for(const auto& contact : matched_contacts) {
        cout << contact.name << " " << contact.number << endl;
    }

    // Cleanup
    free(h_names);
    free(h_results);
    cudaFree(d_names);
    cudaFree(d_results);
    cudaFree(d_search_name);

    return 0;
}

Writing search_phonebook_sort.cu


In [ ]:
!nvcc -arch=sm_75 search_phonebook_sort.cu -o search_phonebook

In [ ]:
!time ./search_phonebook AKTER 100 > output_sort.txt


real	0m0.229s
user	0m0.014s
sys	0m0.211s


In [ ]:
!time ./search_phonebook AKTER 1000 > output_sort.txt


real	0m0.238s
user	0m0.020s
sys	0m0.202s


### Searching & **Sorting Desending** orders

In [ ]:
%%writefile search_phonebook_dsort.cu
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <cstdlib>
#include <cstring>
#include <algorithm>
#include <cuda_runtime.h>

using namespace std;

#define MAX_STR_LEN 50

// Helper struct for sorting on the CPU
struct ResultContact {
    string name;
    string number;

    // CHANGE HERE: Changed < to > for Descending Order
    bool operator<(const ResultContact& other) const {
        return name > other.name;
    }
};

// Substring check logic (GPU)
__device__ bool check(char* str1, char* str2, int len) {
    for(int i = 0; str1[i] != '\0'; i++) {
        int j = 0;
        while(str1[i+j] != '\0' && str2[j] != '\0' && str1[i+j] == str2[j]) {
            j++;
        }
        if(j == len-1) {
            return true;
        }
    }
    return false;
}

// Kernel writes a '1' to the results array if a match is found
__global__ void searchPhonebook(char* d_names, int num_contacts, char* search_name, int name_length, int* d_results) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx < num_contacts) {
        char* current_name = d_names + (idx * MAX_STR_LEN);
        if(check(current_name, search_name, name_length)) {
            d_results[idx] = 1;
        } else {
            d_results[idx] = 0;
        }
    }
}

int main(int argc, char* argv[]) {
    if(argc != 3) {
        cerr << "Usage: " << argv[0] << " <search_name> <num_threads>" << endl;
        return 1;
    }

    string search_string = argv[1];
    int num_threads = atoi(argv[2]);
    string file_name = "phonebook.txt";

    vector<string> host_names_vec;
    vector<string> host_numbers_vec;

    ifstream file(file_name);
    if(!file.is_open()) {
        cerr << "Error opening file: " << file_name << endl;
        return 1;
    }

    string line;
    while(getline(file, line)) {
        if(line.empty()) continue;
        int pos = line.find(",");
        string name = line.substr(1, pos - 2);
        string number = line.substr(pos + 2, line.size() - pos - 3);
        host_names_vec.push_back(name);
        host_numbers_vec.push_back(number);
    }
    file.close();

    int num_contacts = host_names_vec.size();

    char* h_names = (char*)malloc(num_contacts * MAX_STR_LEN);
    int* h_results = (int*)malloc(num_contacts * sizeof(int));

    for(int i = 0; i < num_contacts; i++) {
        strncpy(h_names + (i * MAX_STR_LEN), host_names_vec[i].c_str(), MAX_STR_LEN - 1);
        h_names[(i * MAX_STR_LEN) + (MAX_STR_LEN - 1)] = '\0';
    }

    char *d_names, *d_search_name;
    int *d_results;
    int search_len = search_string.length() + 1;

    cudaMalloc((void**)&d_names, num_contacts * MAX_STR_LEN);
    cudaMalloc((void**)&d_results, num_contacts * sizeof(int));
    cudaMalloc((void**)&d_search_name, search_len);

    cudaMemcpy(d_names, h_names, num_contacts * MAX_STR_LEN, cudaMemcpyHostToDevice);
    cudaMemcpy(d_search_name, search_string.c_str(), search_len, cudaMemcpyHostToDevice);

    int blocks = (num_contacts + num_threads - 1) / num_threads;
    searchPhonebook<<<blocks, num_threads>>>(d_names, num_contacts, d_search_name, search_len, d_results);
    cudaDeviceSynchronize();

    cudaMemcpy(h_results, d_results, num_contacts * sizeof(int), cudaMemcpyDeviceToHost);

    vector<ResultContact> matched_contacts;
    for(int i = 0; i < num_contacts; i++) {
        if(h_results[i] == 1) {
            matched_contacts.push_back({host_names_vec[i], host_numbers_vec[i]});
        }
    }

    // Sorts using the modified operator (Descending)
    sort(matched_contacts.begin(), matched_contacts.end());

    cout << "Search Results (Descending Order):" << endl;
    for(const auto& contact : matched_contacts) {
        cout << contact.name << " " << contact.number << endl;
    }

    free(h_names);
    free(h_results);
    cudaFree(d_names);
    cudaFree(d_results);
    cudaFree(d_search_name);

    return 0;
}

Writing search_phonebook_dsort.cu


In [ ]:
!nvcc -arch=sm_75 search_phonebook_dsort.cu -o search_phonebook

In [ ]:
!time ./search_phonebook AKTER 100 > output_dsort.txt


real	0m0.231s
user	0m0.015s
sys	0m0.209s


In [ ]:
!time ./search_phonebook AKTER 1000 > output_dsort.txt


real	0m0.210s
user	0m0.010s
sys	0m0.194s


## Sub String Searching

In [ ]:
%%writefile substring_search.cu

#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <cstdlib>
#include <cstring>
#include <cuda.h>

using namespace std;

#define MAX_STR_LEN 256
#define MAX_QUERY_LEN 128

// ======================================================
// Device Function: Longest Common Substring (LCSubstr)
// contiguous match only
// ======================================================
__device__ int lcsubstr_length(char* a, int n, char* b, int m)
{
    int dp[MAX_QUERY_LEN + 1][MAX_STR_LEN + 1];
    int max_len = 0;

    // initialize
    for(int i = 0; i <= n; i++)
        for(int j = 0; j <= m; j++)
            dp[i][j] = 0;

    // DP
    for(int i = 1; i <= n; i++) {
        for(int j = 1; j <= m; j++) {

            if(a[i-1] == b[j-1]) {
                dp[i][j] = dp[i-1][j-1] + 1;

                if(dp[i][j] > max_len)
                    max_len = dp[i][j];
            }
            else {
                dp[i][j] = 0;   // difference from LCS
            }
        }
    }

    return max_len;
}


// ======================================================
// Kernel: each thread checks one line
// ======================================================
__global__ void lcsubstrSearch(char* d_lines,
                               int num_lines,
                               char* search,
                               int query_len,
                               int threshold)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx >= num_lines) return;

    char* line = d_lines + idx * MAX_STR_LEN;

    // find line length
    int line_len = 0;
    while(line[line_len] != '\0' && line_len < MAX_STR_LEN)
        line_len++;

    int score = lcsubstr_length(search, query_len, line, line_len);

    if(score >= threshold) {
        printf("Match (LCSubstr=%d): %s\n", score, line);
    }
}



// ======================================================
// MAIN
// ======================================================
int main(int argc, char* argv[])
{
    if(argc != 4) {
        cout << "Usage:\n";
        cout << "./substring_search <search_string> <threads_per_block> <threshold>\n";
        return 0;
    }

    string query = argv[1];
    int threads = atoi(argv[2]);
    int threshold = atoi(argv[3]);
    int query_len = query.size();

    string file_name = "dataset.txt";

    // -----------------------------
    // Read dataset
    // -----------------------------
    vector<string> lines;
    ifstream file(file_name);

    if(!file) {
        cout << "Cannot open dataset.txt\n";
        return 0;
    }

    string line;
    while(getline(file, line)) {
        if(!line.empty())
            lines.push_back(line);
    }

    int n = lines.size();

    if(n == 0){
        cout << "No lines in dataset.txt\n";
        return 0;
    }

    // -----------------------------
    // Flatten memory
    // -----------------------------
    char* h_lines = (char*)malloc(n * MAX_STR_LEN);

    for(int i=0;i<n;i++){
        strncpy(h_lines + i*MAX_STR_LEN, lines[i].c_str(), MAX_STR_LEN-1);
        h_lines[i*MAX_STR_LEN + MAX_STR_LEN-1] = '\0';
    }

    // -----------------------------
    // GPU memory
    // -----------------------------
    char *d_lines, *d_query;

    cudaMalloc(&d_lines, n * MAX_STR_LEN);
    cudaMalloc(&d_query, query_len + 1);

    cudaMemcpy(d_lines, h_lines, n * MAX_STR_LEN, cudaMemcpyHostToDevice);
    cudaMemcpy(d_query, query.c_str(), query_len + 1, cudaMemcpyHostToDevice);

    // -----------------------------
    // Kernel launch
    // -----------------------------
    int blocks = (n + threads - 1) / threads;

    lcsubstrSearch<<<blocks, threads>>>(
        d_lines,
        n,
        d_query,
        query_len,
        threshold
    );

    cudaDeviceSynchronize();

    // -----------------------------
    // Cleanup
    // -----------------------------
    cudaFree(d_lines);
    cudaFree(d_query);
    free(h_lines);

    return 0;
}


Writing substring_search.cu


In [ ]:
!nvcc -O2 -arch=sm_75 substring_search.cu -o substring_search


In [ ]:
!time ./substring_search search 256 2 > output_LCSubstr.txt



real	0m0.232s
user	0m0.016s
sys	0m0.211s


## Sub-Sequence Searching

In [ ]:
%%writefile subsequence_search.cu
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <cstdlib>
#include <cstring>
#include <cuda.h>

using namespace std;

#define MAX_STR_LEN 256
#define MAX_QUERY_LEN 128

// =====================================
// LCS Device Function (DP algorithm)
// =====================================
__device__ int lcs_length(char* a, int n, char* b, int m) {
    int dp[MAX_QUERY_LEN + 1][MAX_STR_LEN + 1];

    for(int i=0;i<=n;i++)
        for(int j=0;j<=m;j++)
            dp[i][j] = 0;

    for(int i=1;i<=n;i++) {
        for(int j=1;j<=m;j++) {
            if(a[i-1] == b[j-1])
                dp[i][j] = dp[i-1][j-1] + 1;
            else
                dp[i][j] = max(dp[i-1][j], dp[i][j-1]);
        }
    }

    return dp[n][m];
}

// =====================================
// Kernel: LCS Search
// =====================================
__global__ void lcsSearch(char* d_lines,
                          int num_lines,
                          char* search,
                          int query_len,
                          int threshold) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if(idx >= num_lines) return;

    char* line = d_lines + idx * MAX_STR_LEN;

    // compute line length on device
    int line_len = 0;
    while(line[line_len] != '\0' && line_len < MAX_STR_LEN) line_len++;

    int lcs = lcs_length(search, query_len, line, line_len);

    if(lcs >= threshold) {
        printf("Match (LCS=%d): %s\n", lcs, line);
    }
}

// =====================================
// MAIN
// =====================================
int main(int argc, char* argv[]) {

    if(argc != 4) {
        cout << "Usage: ./substring_search <search_string> <threads> <threshold>\n";
        return 0;
    }

    string query = argv[1];
    int threads = atoi(argv[2]);
    int threshold = atoi(argv[3]);
    int query_len = query.size();

    string file_name = "dataset.txt";

    // Read file
    vector<string> lines;
    ifstream file(file_name);

    string line;
    while(getline(file, line)) {
        if(!line.empty())
            lines.push_back(line);
    }

    int n = lines.size();
    if(n == 0){
        cout << "No lines in dataset.txt\n";
        return 0;
    }

    // Flatten memory
    char* h_lines = (char*)malloc(n * MAX_STR_LEN);

    for(int i=0;i<n;i++){
        strncpy(h_lines + i*MAX_STR_LEN, lines[i].c_str(), MAX_STR_LEN-1);
        h_lines[i*MAX_STR_LEN + MAX_STR_LEN-1] = '\0';
    }

    // Device memory
    char *d_lines, *d_query;

    cudaMalloc(&d_lines, n * MAX_STR_LEN);
    cudaMalloc(&d_query, query_len + 1);

    cudaMemcpy(d_lines, h_lines, n * MAX_STR_LEN, cudaMemcpyHostToDevice);
    cudaMemcpy(d_query, query.c_str(), query_len + 1, cudaMemcpyHostToDevice);

    // Launch kernel
    int blocks = (n + threads - 1) / threads;
    lcsSearch<<<blocks, threads>>>(d_lines, n, d_query, query_len, threshold);

    cudaDeviceSynchronize();

    cudaFree(d_lines);
    cudaFree(d_query);
    free(h_lines);

    return 0;
}


In [ ]:
!nvcc -arch=sm_75 subsequence_search.cu -o subsequence_search

In [ ]:
## ./subsequence_search <search_string> <threads> <threshold(minimum lcs length)>
!time ./subsequence_search search 128 4 > lcs.txt


real	0m0.240s
user	0m0.018s
sys	0m0.217s


In [ ]:
!time ./substring_search search 256 6 > lcs.txt


real	0m0.243s
user	0m0.011s
sys	0m0.220s


## Word Count Sort

In [10]:
%%writefile fruit_count_cuda.cu
#include <iostream>
#include <fstream>
#include <vector>
#include <string>
#include <algorithm>
#include <cuda.h>
#include <cstring>

using namespace std;

#define MAX_WORD_LEN 32

// CUDA Kernel: count words using atomic operations
__global__ void countWordsKernel(char *words, int n, char *unique, int uniqueCount, int *counts) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        char *currentWord = words + idx * MAX_WORD_LEN;
        for (int i = 0; i < uniqueCount; i++) {
            char *uniqWord = unique + i * MAX_WORD_LEN;
            bool match = true;
            for (int j = 0; j < MAX_WORD_LEN; j++) {
                if (currentWord[j] != uniqWord[j]) {
                    match = false;
                    break;
                }
                if (currentWord[j] == '\0') break;
            }
            if (match) {
                atomicAdd(&counts[i], 1);
                break;
            }
        }
    }
}

int main() {
    // Step 1: Read file
    ifstream file("input.txt");
    if (!file.is_open()) {
        cerr << "Error opening file!" << endl;
        return 1;
    }

    vector<string> wordList;
    string word;
    while (file >> word) wordList.push_back(word);
    file.close();

    if (wordList.empty()) {
        cerr << "File is empty!" << endl;
        return 1;
    }

    // Step 2: Build unique words
    vector<string> uniqueWords;
    for (auto &w : wordList) {
        if (find(uniqueWords.begin(), uniqueWords.end(), w) == uniqueWords.end())
            uniqueWords.push_back(w);
    }

    int n = wordList.size();
    int uniqueCount = uniqueWords.size();

    // Step 3: Flatten arrays for GPU
    char *h_words = new char[n * MAX_WORD_LEN];
    char *h_unique = new char[uniqueCount * MAX_WORD_LEN];
    for (int i = 0; i < n; i++) {
        strncpy(&h_words[i * MAX_WORD_LEN], wordList[i].c_str(), MAX_WORD_LEN);
        h_words[i * MAX_WORD_LEN + MAX_WORD_LEN - 1] = '\0';
    }
    for (int i = 0; i < uniqueCount; i++) {
        strncpy(&h_unique[i * MAX_WORD_LEN], uniqueWords[i].c_str(), MAX_WORD_LEN);
        h_unique[i * MAX_WORD_LEN + MAX_WORD_LEN - 1] = '\0';
    }

    // Step 4: Allocate device memory
    char *d_words, *d_unique;
    int *d_counts;
    cudaMalloc(&d_words, n * MAX_WORD_LEN);
    cudaMalloc(&d_unique, uniqueCount * MAX_WORD_LEN);
    cudaMalloc(&d_counts, uniqueCount * sizeof(int));
    cudaMemcpy(d_words, h_words, n * MAX_WORD_LEN, cudaMemcpyHostToDevice);
    cudaMemcpy(d_unique, h_unique, uniqueCount * MAX_WORD_LEN, cudaMemcpyHostToDevice);
    cudaMemset(d_counts, 0, uniqueCount * sizeof(int));

    // Step 5: Launch kernel
    int threads = 256;
    int blocks = (n + threads - 1) / threads;
    countWordsKernel<<<blocks, threads>>>(d_words, n, d_unique, uniqueCount, d_counts);
    cudaDeviceSynchronize();

    // Step 6: Copy results back
    int *h_counts = new int[uniqueCount];
    cudaMemcpy(h_counts, d_counts, uniqueCount * sizeof(int), cudaMemcpyDeviceToHost);

    // Step 7: Sort and print top 10
    vector<pair<string,int>> freq;
    for (int i = 0; i < uniqueCount; i++)
        freq.push_back({uniqueWords[i], h_counts[i]});

    sort(freq.begin(), freq.end(), [](auto &a, auto &b){ return b.second > a.second; });

    cout << "Top 10 fruit occurrences:\n";
    for (int i = 0; i < min(10, (int)freq.size()); i++)
        cout << freq[i].first << " : " << freq[i].second << endl;

    // Step 8: Free memory
    delete[] h_words;
    delete[] h_unique;
    delete[] h_counts;
    cudaFree(d_words);
    cudaFree(d_unique);
    cudaFree(d_counts);

    return 0;
}


Overwriting fruit_count_cuda.cu


In [11]:
!nvcc -o fruit_count_cuda fruit_count_cuda.cu


In [12]:
!./fruit_count_cuda

Top 10 fruit occurrences:
Lychee : 0
Peach : 0
Plum : 0
Cherry : 0
Apricot : 0
Honeydew : 0
Cantaloupe : 0
Watermelon : 0
Guava : 0
Jackfruit : 0
